In [2]:
import numpy as np

In [153]:
number_of_features = 3
number_of_outputs = 2

hidden_layers = [] # number of nodes in each layer
hidden_layers.append(2)

weight_matrices = []
__biases = []

for i in range(len(hidden_layers)):
    matrix = None
    bias = np.zeros((hidden_layers[i], 1), dtype=np.float64)
    if i == 0:
        matrix = np.zeros((number_of_features, hidden_layers[i]), dtype=np.float64)
    else:
        matrix = np.zeros((hidden_layers[i-1], hidden_layers[i]), dtype=np.float64)
    
    weight_matrices.append(matrix)
    __biases.append(bias)
    
matrix = np.zeros((hidden_layers[-1], number_of_outputs), dtype=np.float64)
bias = np.zeros((number_of_outputs, 1), dtype=np.float64)

weight_matrices.append(matrix)
__biases.append(bias)

In [159]:

y_out = forwardPass(x_input)
print(y_out)

[[0.88955061]
 [0.80039961]]


In [3]:
from abc import abstractmethod

class ActivationFunction:
    def __init__(self) -> None:
        pass
    
    @abstractmethod
    def activate(self, input_z):
        pass
    
    @abstractmethod
    def gradient(self, input_z):
        pass

In [8]:
class SigmoidActivation(ActivationFunction):
    def activate(self, input):
        return 1.0 / (1 + np.exp(-input))
    
    def gradient(self, input):
        sig = 1.0 / (1 + np.exp(-input))
        return (1.0 - sig) * sig

In [18]:
class Layer:
    def __init__(self, number_of_nodes : int, activation_function : ActivationFunction = None):
        self.__number_of_nodes = number_of_nodes
        self.__activation_function : ActivationFunction = activation_function
    
    @property
    def number_of_nodes(self):
        return self.__number_of_nodes
    
    
    def activate(self, input):
        if self.__activation_function:
            return self.__activation_function.activate(input)
        
        return input
    
    def gradient(self, input):
        if self.__activation_function:
            return self.__activation_function.gradient(input)
        
        print("No Activation Function is added...")
        return np.ones(input.shape, dtype=np.float64)

In [143]:
class NeuralNetwork:
    def __init__(self, number_of_inputs, number_of_outputs, output_activation_function : ActivationFunction = None) -> None:
        self.__number_of_inputs = number_of_inputs
        self.__number_of_outputs = number_of_outputs
        self.__hidden_layers : list[Layer] = []
        self.__output_layer : Layer = None
        if output_activation_function:
            self.__output_layer = Layer(number_of_outputs, output_activation_function)
        self.__is_initialized = False
    
    def addHiddenLayer(self, layer : Layer):
        self.__is_initialized = False
        self.__hidden_layers.append(layer)
        
    def initialize(self):
        self.__is_initialized = True
        self.__weights = []
        self.__biases = []
        self.__weighted_sum = []
        self.__activated_output = [] # output after applying activation function in each layer
        self.__delta_error = [] # error in each layer
        
        prev_layer_size = self.__number_of_inputs
        for i in range(len(self.__hidden_layers)):
            self.__weights.append(np.zeros((prev_layer_size, self.__hidden_layers[i].number_of_nodes), dtype=np.float64))
            self.__biases.append(np.zeros((self.__hidden_layers[i].number_of_nodes, 1), dtype=np.float64))
            self.__weighted_sum.append(np.zeros((self.__hidden_layers[i].number_of_nodes, 1), dtype=np.float64))
            self.__activated_output.append(np.zeros((self.__hidden_layers[i].number_of_nodes, 1), dtype=np.float64))
            self.__delta_error.append(np.zeros((self.__hidden_layers[i].number_of_nodes, 1), dtype=np.float64))
            prev_layer_size = self.__hidden_layers[i].number_of_nodes
            
        self.__weights.append(np.zeros((prev_layer_size, self.__number_of_outputs), dtype=np.float64))
        self.__biases.append(np.zeros((self.__number_of_outputs, 1), dtype=np.float64))
        self.__weighted_sum.append(np.zeros((self.__number_of_outputs, 1), dtype=np.float64))
        self.__activated_output.append(np.zeros((self.__number_of_outputs, 1), dtype=np.float64))
        self.__delta_error.append(np.zeros((self.__number_of_outputs, 1), dtype=np.float64))
    
    @property
    def weights(self):
        return self.__weights
    
    @property
    def biases(self):
        return self.__biases
    
    @property
    def activated_output(self):
        return self.__activated_output
    
    def initializeWeights(self, weights):
        self.__weights = weights
        
    def initializeBiases(self, biases):
        self.__biases = biases
    
    def forwardPass(self, input):
        if not self.__is_initialized:
            self.initialize()
        
        for i in range(len(self.__hidden_layers) + 1):
            if i == 0:
                self.__weighted_sum[i] = np.matmul(self.__weights[i].T, input) + self.__biases[i]
                self.__activated_output[i] = self.__hidden_layers[i].activate(self.__weighted_sum[i])
            elif i == len(self.__hidden_layers):
                self.__weighted_sum[i] = np.matmul(self.__weights[i].T, self.__activated_output[i-1]) + self.__biases[i]
                if self.__output_layer:
                    self.__activated_output[i] = self.__output_layer.activate(self.__weighted_sum[i])
                else:
                    self.__activated_output[i] = self.__weighted_sum[i]
            else:
                self.__weighted_sum[i] = np.matmul(self.__weights[i].T, self.__activated_output[i-1]) + self.__biases[i]
                self.__activated_output[i] = self.__hidden_layers[i].activate(self.__weighted_sum[i])
            
            # print(self.__weighted_sum[i])
            # print(self.__activated_output[i])
    
    def backpropagate(self, output):
        pass

In [144]:
nn = NeuralNetwork(3, 2, SigmoidActivation())

nn.addHiddenLayer(Layer(2, SigmoidActivation()))

nn.initialize()

weights = [np.array([[0.1, 0.2],
                     [0.3, 0.4],
                     [0.5, 0.6]]),
            np.array([[0.7, 0.8],
                      [0.9, 0.1]])]

biases = [np.array([[0.5],
                    [0.5]]),
            np.array([[0.5],
                        [0.5]])]

nn.initializeWeights(weights)
nn.initializeBiases(biases)

In [145]:
for w in nn.weights:
    print(w)
    
for b in nn.biases:
    print(b)

[[0.1 0.2]
 [0.3 0.4]
 [0.5 0.6]]
[[0.7 0.8]
 [0.9 0.1]]
[[0.5]
 [0.5]]
[[0.5]
 [0.5]]


In [146]:
x_input = np.array([[1.0],[4.0],[5.0]], dtype=np.float16)

nn.forwardPass(x_input)

In [147]:
nn.activated_output[-1]

array([[0.88955061],
       [0.80039961]])